In [1]:
import cv2
import mediapipe as mp
import numpy as np
import math
from tkinter import *
from tkinter import ttk
from tkinter.ttk import Style

class ExerciseCounter:
    def __init__(self):
        self.exercise_type = None
        self.counter = 0
        self.stage = None
        self.running = True
        self.form_feedback = ""
        
        # Exercise definitions with updated exercises
        self.exercise_info = {
            'Bicep Curl': {
                'description': 'Classic arm exercise for bicep strength',
                'steps': [
                    '1. Stand straight, dumbbells at sides',
                    '2. Curl weights up to shoulders',
                    '3. Lower slowly back down',
                    '4. Keep elbows fixed at sides'
                ],
                'position': 'Stand FACING the camera',
                'landmarks': ['LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST',
                            'RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST']
            },
            'Lateral Raise': {
                'description': 'Shoulder exercise for deltoid development',
                'steps': [
                    '1. Stand straight, dumbbells at sides',
                    '2. Raise arms to shoulder level',
                    '3. Keep slight bend in elbows',
                    '4. Lower controlled to start'
                ],
                'position': 'Stand FACING the camera',
                'landmarks': ['LEFT_HIP', 'LEFT_SHOULDER', 'LEFT_ELBOW',
                            'RIGHT_HIP', 'RIGHT_SHOULDER', 'RIGHT_ELBOW']
            },
            'Tricep Pushdown': {
                'description': 'Isolation exercise for tricep development',
                'steps': [
                    '1. Stand with elbows tucked at sides',
                    '2. Start with forearms parallel to ground',
                    '3. Push hands down until arms fully extended',
                    '4. Slowly return to starting position'
                ],
                'position': 'Stand FACING the camera',
                'landmarks': ['LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST',
                            'RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST']
            },
            'Shoulder Press': {
                'description': 'Vertical pressing movement for shoulder strength',
                'steps': [
                    '1. Start with weights at shoulder level',
                    '2. Press weights overhead until arms are extended',
                    '3. Keep core tight and avoid arching back',
                    '4. Lower weights back to shoulder level'
                ],
                'position': 'Stand FACING the camera',
                'landmarks': ['LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST',
                             'RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST']
            },
            'Front Raise': {
                'description': 'Isolation exercise for anterior deltoids',
                'steps': [
                    '1. Stand with weights in front of thighs',
                    '2. Raise arms straight in front to shoulder height',
                    '3. Keep slight bend in elbows',
                    '4. Lower slowly back to start'
                ],
                'position': 'Stand FACING the camera',
                'landmarks': ['LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST',
                             'RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST']
            },
            'Lat Pulldown': {
                'description': 'Compound exercise for back width and strength',
                'steps': [
                    '1. Sit facing the camera with arms extended overhead',
                    '2. Pull hands down toward shoulders',
                    '3. Keep chest up and shoulders back',
                    '4. Slowly return to starting position'
                ],
                'position': 'Sit FACING the camera',
                'landmarks': ['LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST',
                             'RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST']
            },
            'Seated Row': {
                'description': 'Compound exercise for mid-back strength',
                'steps': [
                    '1. Sit facing the camera with arms extended forward',
                    '2. Pull hands toward torso, elbows close to body',
                    '3. Squeeze shoulder blades together',
                    '4. Slowly extend arms back to start'
                ],
                'position': 'Sit FACING the camera',
                'landmarks': ['LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST',
                             'RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST']
            },
            'Leg Curl': {
                'description': 'Isolation exercise for hamstring development',
                'steps': [
                    '1. Stand sideways to camera',
                    '2. Bend knee to bring heel toward buttocks',
                    '3. Keep thigh stationary throughout movement',
                    '4. Lower foot back to starting position with control'
                ],
                'position': 'Stand SIDEWAYS to the camera',
                'landmarks': ['LEFT_HIP', 'LEFT_KNEE', 'LEFT_ANKLE',
                             'RIGHT_HIP', 'RIGHT_KNEE', 'RIGHT_ANKLE']
            },
            'Leg Extension': {
                'description': 'Isolation exercise for quadricep strength',
                'steps': [
                    '1. Sit sideways to camera',
                    '2. Extend knee to straighten leg',
                    '3. Hold briefly at full extension',
                    '4. Lower leg back to 90-degree position'
                ],
                'position': 'Sit SIDEWAYS to the camera',
                'landmarks': ['LEFT_HIP', 'LEFT_KNEE', 'LEFT_ANKLE',
                             'RIGHT_HIP', 'RIGHT_KNEE', 'RIGHT_ANKLE']
            },
            'Abdominal Crunch': {
                'description': 'Core exercise targeting rectus abdominis',
                'steps': [
                    '1. Lie on back with knees bent, sideways to camera',
                    '2. Place hands lightly behind or beside head',
                    '3. Curl upper body toward knees',
                    '4. Lower back down with control'
                ],
                'position': 'Lie SIDEWAYS to the camera',
                'landmarks': ['LEFT_SHOULDER', 'LEFT_HIP', 'LEFT_KNEE',
                             'RIGHT_SHOULDER', 'RIGHT_HIP', 'RIGHT_KNEE']
            },
            'Chest Fly': {
                'description': 'Isolation exercise for chest muscles',
                'steps': [
                    '1. Stand with arms extended to sides at shoulder height',
                    '2. Bring arms together in front of chest',
                    '3. Feel squeeze in chest muscles',
                    '4. Return to starting position with control'
                ],
                'position': 'Stand FACING the camera',
                'landmarks': ['LEFT_SHOULDER', 'LEFT_ELBOW', 'LEFT_WRIST',
                             'RIGHT_SHOULDER', 'RIGHT_ELBOW', 'RIGHT_WRIST']
            },
            'Shrugs': {
                'description': 'Isolation exercise for trapezius muscles',
                'steps': [
                    '1. Stand holding weights at sides',
                    '2. Elevate shoulders toward ears as high as possible',
                    '3. Hold briefly at top position',
                    '4. Lower shoulders with control'
                ],
                'position': 'Stand FACING the camera',
                'landmarks': ['LEFT_SHOULDER', 'LEFT_EAR', 'RIGHT_SHOULDER', 'RIGHT_EAR']
            }
        }
        
        self.create_main_window()

    def validate_form(self, exercise_type, landmarks, angles):
        """Validate exercise form and return feedback"""
        try:
            if exercise_type == 'Bicep Curl':
                left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                
                # Check if arms are too far from body
                if abs(left_shoulder[0] - landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x) > 0.1:
                    return "Keep left elbow closer to body"
                if abs(right_shoulder[0] - landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x) > 0.1:
                    return "Keep right elbow closer to body"
                
                # Check if arms are moving together
                if abs(angles['left'] - angles['right']) > 20:
                    return "Keep arms moving together"
                
                return ""
                
            elif exercise_type == 'Lateral Raise':
                # Check if arms are raised too high
                if angles['left'] > 100 or angles['right'] > 100:
                    return "Don't raise arms above shoulder level"
                
                # Check if arms are moving together
                if abs(angles['left'] - angles['right']) > 15:
                    return "Keep arms moving together"
                
                return ""
                
            elif exercise_type == 'Tricep Pushdown':
                left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                
                # Check if elbows are tucked
                if abs(left_shoulder[0] - landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x) > 0.1:
                    return "Keep left elbow tucked at side"
                if abs(right_shoulder[0] - landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x) > 0.1:
                    return "Keep right elbow tucked at side"
                
                # Check if arms are moving together
                if abs(angles['left'] - angles['right']) > 20:
                    return "Keep arms moving together"
                
                return ""
                
            elif exercise_type == 'Shoulder Press':
                # Check if arms are aligned with shoulders
                left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                
                left_elbow = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y]
                right_elbow = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x,
                             landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].y]
                
                # Check horizontal alignment
                if abs(left_elbow[0] - left_shoulder[0]) > 0.1:
                    return "Keep left elbow aligned with shoulder"
                if abs(right_elbow[0] - right_shoulder[0]) > 0.1:
                    return "Keep right elbow aligned with shoulder"
                
                # Check if arms are moving together
                if abs(angles['left'] - angles['right']) > 20:
                    return "Keep arms moving together"
                    
                return ""
                
            elif exercise_type == 'Front Raise':
                # Check if arms are raised too high
                if angles['left'] > 100 or angles['right'] > 100:
                    return "Don't raise arms above shoulder level"
                
                # Check if arms are moving together
                if abs(angles['left'] - angles['right']) > 15:
                    return "Keep arms moving together"
                
                return ""
                
            elif exercise_type == 'Lat Pulldown':
                # Check if elbows are moving down and back
                left_elbow = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y]
                right_elbow = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x,
                             landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].y]
                
                # Check width between elbows (shouldn't be too narrow)
                elbow_width = abs(left_elbow[0] - right_elbow[0])
                if elbow_width < 0.2:
                    return "Keep elbows wider apart"
                
                # Check if arms are moving together
                if abs(angles['left'] - angles['right']) > 15:
                    return "Keep arms moving together"
                
                return ""
                
            elif exercise_type == 'Seated Row':
                # Check if elbows are moving back close to body
                left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
                right_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                
                left_elbow = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].x,
                            landmarks[mp.solutions.pose.PoseLandmark.LEFT_ELBOW.value].y]
                right_elbow = [landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].x,
                             landmarks[mp.solutions.pose.PoseLandmark.RIGHT_ELBOW.value].y]
                
                # Check if elbows are too wide at the end of movement
                if left_elbow[0] < left_shoulder[0] - 0.1:
                    return "Keep left elbow closer to body"
                if right_elbow[0] > right_shoulder[0] + 0.1:
                    return "Keep right elbow closer to body"
                
                # Check if arms are moving together
                if abs(angles['left'] - angles['right']) > 15:
                    return "Keep arms moving together"
                
                return ""
                
            elif exercise_type == 'Leg Curl':
                # Check for isolated hamstring movement
                left_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
                          landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
                left_knee = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].x,
                           landmarks[mp.solutions.pose.PoseLandmark.LEFT_KNEE.value].y]
                
                # Check if hip is moving (should be stable)
                if abs(left_hip[1] - self.last_hip_pos[1]) > 0.03:
                    return "Keep hip stable, isolate leg movement"
                
                # Check knee flex angle
                if angles['left'] < 30:
                    return "Increase range of motion"
                
                return ""
                
            elif exercise_type == 'Leg Extension':
                # Check for isolated quad movement
                left_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
                          landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
                
                # Check if hip is moving (should be stable)
                if abs(left_hip[1] - self.last_hip_pos[1]) > 0.03:
                    return "Keep hip stable, isolate leg movement"
                
                # Check leg extension angle
                if angles['left'] < 160:
                    return "Extend leg more completely"
                
                return ""
                
            elif exercise_type == 'Abdominal Crunch':
                # Check for proper form
                left_shoulder = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                               landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y]
                left_hip = [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
                          landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
                
                # Measure curl up distance
                curl_distance = self.initial_shoulder_hip_dist - abs(left_shoulder[1] - left_hip[1])
                
                # Check if movement is too small
                if curl_distance < 0.05:
                    return "Increase range of motion"
                
                # Check if user is pulling with neck rather than abs
                neck_angle = self.calculate_angle(
                    [landmarks[mp.solutions.pose.PoseLandmark.LEFT_EAR.value].x,
                     landmarks[mp.solutions.pose.PoseLandmark.LEFT_EAR.value].y],
                    [landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y],
                    [landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].x,
                     landmarks[mp.solutions.pose.PoseLandmark.LEFT_HIP.value].y]
                )
                
                if neck_angle < 130:
                    return "Don't pull with neck, use abs"
                
                return ""
                
            elif exercise_type == 'Chest Fly':
                # Check if arms are at shoulder height
                left_shoulder_y = landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y
                right_shoulder_y = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y
                left_wrist_y = landmarks[mp.solutions.pose.PoseLandmark.LEFT_WRIST.value].y
                right_wrist_y = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_WRIST.value].y
                
                if abs(left_shoulder_y - left_wrist_y) > 0.1:
                    return "Keep arms at shoulder height"
                if abs(right_shoulder_y - right_wrist_y) > 0.1:
                    return "Keep arms at shoulder height"
                
                # Check if arms are moving together
                if abs(angles['left'] - angles['right']) > 15:
                    return "Keep arms moving together"
                
                return ""
                
            elif exercise_type == 'Shrugs':
                # Check if shoulders are moving up
                left_shoulder_y = landmarks[mp.solutions.pose.PoseLandmark.LEFT_SHOULDER.value].y
                right_shoulder_y = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER.value].y
                left_ear_y = landmarks[mp.solutions.pose.PoseLandmark.LEFT_EAR.value].y
                right_ear_y = landmarks[mp.solutions.pose.PoseLandmark.RIGHT_EAR.value].y
                
                shoulder_ear_dist = abs((left_shoulder_y - left_ear_y) + (right_shoulder_y - right_ear_y)) / 2
                if shoulder_ear_dist > 0.15:
                    return "Raise shoulders higher"
                
                return ""
                
        except Exception as e:
            # Return empty feedback if there's an error rather than crashing
            return ""

    def calculate_angle(self, a, b, c):
        try:
            a = np.array(a)
            b = np.array(b)
            c = np.array(c)
            radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
            angle = np.abs(radians*180.0/np.pi)
            if angle > 180.0:
                angle = 360-angle
            return angle
        except:
            return 0  # Return 0 instead of crashing

    def start_camera(self):
        mp_pose = mp.solutions.pose
        pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
        mp_draw = mp.solutions.drawing_utils
        
        cap = cv2.VideoCapture(0)
        # Set more realistic resolution to reduce processing load
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        
        window_name = 'Exercise Counter'
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        
        # For efficient path drawing in exercises
        path_timer = 0
        show_path = False
        
        # Initialize tracking variables for the new exercises
        self.last_hip_pos = [0, 0]
        self.initial_shoulder_hip_dist = 0
        self.movement_history = []  # For trajectory tracking
        
        # For tracking movement patterns in 3D space
        movement_buffer = []
        buffer_size = 10
        
        while cap.isOpened() and self.running:
            ret, frame = cap.read()
            if not ret:
                break
                
            # Resize frame for faster processing (reduce lag)
            frame = cv2.resize(frame, (960, 540))
            
            # Convert to RGB for MediaPipe
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            overlay = image.copy()
            
            if results.pose_landmarks:
                landmarks = results.pose_landmarks.landmark
                
                # Initialize angles with default values to prevent crashes
                angles = {'left': 0, 'right': 0}
                
                try:
                    h, w, c = image.shape
                    
                    # Store hip position for stability checks
                    self.last_hip_pos = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                    
                    # Calculate shoulder-hip distance for ab exercises
                    self.initial_shoulder_hip_dist = abs(
                        landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y - 
                        landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y
                    )
                    
                    if self.exercise_type == 'Bicep Curl':
                        # Left arm angle
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
                        angles['left'] = self.calculate_angle(left_shoulder, left_elbow, left_wrist)
                        
                        # Right arm angle
                        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                        
                        angles['right'] = self.calculate_angle(right_shoulder, right_elbow, right_wrist)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_shoulder[0] * w), int(left_shoulder[1] * h)),
                               (int(left_elbow[0] * w), int(left_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(left_elbow[0] * w), int(left_elbow[1] * h)),
                               (int(left_wrist[0] * w), int(left_wrist[1] * h)), (0, 255, 0), 2)
                        
                        cv2.line(image, (int(right_shoulder[0] * w), int(right_shoulder[1] * h)),
                               (int(right_elbow[0] * w), int(right_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(right_elbow[0] * w), int(right_elbow[1] * h)),
                               (int(right_wrist[0] * w), int(right_wrist[1] * h)), (0, 255, 0), 2)
                        
                        # Validate form and update counter
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:  # Only count rep if form is good
                            if angles['left'] > 160 and angles['right'] > 160:
                                self.stage = "down"
                            if angles['left'] < 60 and angles['right'] < 60 and self.stage == "down":
                                self.stage = "up"
                                self.counter += 1
                    
                    elif self.exercise_type == 'Lateral Raise':
                        # Calculate angles for both arms
                        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                        
                        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                        
                        angles['left'] = self.calculate_angle(left_hip, left_shoulder, left_elbow)
                        angles['right'] = self.calculate_angle(right_hip, right_shoulder, right_elbow)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_hip[0] * w), int(left_hip[1] * h)),
                               (int(left_shoulder[0] * w), int(left_shoulder[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(left_shoulder[0] * w), int(left_shoulder[1] * h)),
                               (int(left_elbow[0] * w), int(left_elbow[1] * h)), (0, 255, 0), 2)
                        
                        cv2.line(image, (int(right_hip[0] * w), int(right_hip[1] * h)),
                               (int(right_shoulder[0] * w), int(right_shoulder[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(right_shoulder[0] * w), int(right_shoulder[1] * h)),
                               (int(right_elbow[0] * w), int(right_elbow[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if angles['left'] < 30 and angles['right'] < 30:
                                self.stage = "down"
                            if angles['left'] > 85 and angles['right'] > 85 and self.stage == "down":
                                self.stage = "up"
                                self.counter += 1
                    
                    elif self.exercise_type == 'Tricep Pushdown':
                        # Calculate angles for both arms
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
                        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                        
                        angles['left'] = self.calculate_angle(left_shoulder, left_elbow, left_wrist)
                        angles['right'] = self.calculate_angle(right_shoulder, right_elbow, right_wrist)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_shoulder[0] * w), int(left_shoulder[1] * h)),
                               (int(left_elbow[0] * w), int(left_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(left_elbow[0] * w), int(left_elbow[1] * h)),
                               (int(left_wrist[0] * w), int(left_wrist[1] * h)), (0, 255, 0), 2)
                        
                        cv2.line(image, (int(right_shoulder[0] * w), int(right_shoulder[1] * h)),
                               (int(right_elbow[0] * w), int(right_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(right_elbow[0] * w), int(right_elbow[1] * h)),
                               (int(right_wrist[0] * w), int(right_wrist[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if angles['left'] < 60 and angles['right'] < 60:
                                self.stage = "up"
                            if angles['left'] > 150 and angles['right'] > 150 and self.stage == "up":
                                self.stage = "down"
                                self.counter += 1
                                
                    elif self.exercise_type == 'Shoulder Press':
                        # Calculate angles for both arms
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
                        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                        
                        angles['left'] = self.calculate_angle(left_shoulder, left_elbow, left_wrist)
                        angles['right'] = self.calculate_angle(right_shoulder, right_elbow, right_wrist)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_shoulder[0] * w), int(left_shoulder[1] * h)),
                               (int(left_elbow[0] * w), int(left_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(left_elbow[0] * w), int(left_elbow[1] * h)),
                               (int(left_wrist[0] * w), int(left_wrist[1] * h)), (0, 255, 0), 2)
                        
                        cv2.line(image, (int(right_shoulder[0] * w), int(right_shoulder[1] * h)),
                               (int(right_elbow[0] * w), int(right_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(right_elbow[0] * w), int(right_elbow[1] * h)),
                               (int(right_wrist[0] * w), int(right_wrist[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if angles['left'] < 60 and angles['right'] < 60:
                                self.stage = "down"
                            if angles['left'] > 160 and angles['right'] > 160 and self.stage == "down":
                                self.stage = "up"
                                self.counter += 1
                                
                    elif self.exercise_type == 'Front Raise':
                        # Calculate angles for both arms relative to torso
                        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
                        right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
                        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                        
                        angles['left'] = self.calculate_angle(left_hip, left_shoulder, left_wrist)
                        angles['right'] = self.calculate_angle(right_hip, right_shoulder, right_wrist)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_shoulder[0] * w), int(left_shoulder[1] * h)),
                               (int(left_wrist[0] * w), int(left_wrist[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(right_shoulder[0] * w), int(right_shoulder[1] * h)),
                               (int(right_wrist[0] * w), int(right_wrist[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if angles['left'] < 30 and angles['right'] < 30:
                                self.stage = "down"
                            if angles['left'] > 85 and angles['right'] > 85 and self.stage == "down":
                                self.stage = "up"
                                self.counter += 1
                                
                    elif self.exercise_type == 'Lat Pulldown':
                        # Calculate angles for both arms
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
                        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                        
                        angles['left'] = self.calculate_angle(left_shoulder, left_elbow, left_wrist)
                        angles['right'] = self.calculate_angle(right_shoulder, right_elbow, right_wrist)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_shoulder[0] * w), int(left_shoulder[1] * h)),
                               (int(left_elbow[0] * w), int(left_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(left_elbow[0] * w), int(left_elbow[1] * h)),
                               (int(left_wrist[0] * w), int(left_wrist[1] * h)), (0, 255, 0), 2)
                        
                        cv2.line(image, (int(right_shoulder[0] * w), int(right_shoulder[1] * h)),
                               (int(right_elbow[0] * w), int(right_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(right_elbow[0] * w), int(right_elbow[1] * h)),
                               (int(right_wrist[0] * w), int(right_wrist[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if angles['left'] > 150 and angles['right'] > 150:
                                self.stage = "up"
                            if angles['left'] < 80 and angles['right'] < 80 and self.stage == "up":
                                self.stage = "down"
                                self.counter += 1
                                
                    elif self.exercise_type == 'Seated Row':
                        # Calculate angles for both arms
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
                        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
                        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                        right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
                        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                        
                        angles['left'] = self.calculate_angle(left_shoulder, left_elbow, left_wrist)
                        angles['right'] = self.calculate_angle(right_shoulder, right_elbow, right_wrist)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_shoulder[0] * w), int(left_shoulder[1] * h)),
                               (int(left_elbow[0] * w), int(left_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(left_elbow[0] * w), int(left_elbow[1] * h)),
                               (int(left_wrist[0] * w), int(left_wrist[1] * h)), (0, 255, 0), 2)
                        
                        cv2.line(image, (int(right_shoulder[0] * w), int(right_shoulder[1] * h)),
                               (int(right_elbow[0] * w), int(right_elbow[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(right_elbow[0] * w), int(right_elbow[1] * h)),
                               (int(right_wrist[0] * w), int(right_wrist[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if angles['left'] > 140 and angles['right'] > 140:
                                self.stage = "start"
                            if angles['left'] < 80 and angles['right'] < 80 and self.stage == "start":
                                self.stage = "pulled"
                                self.counter += 1
                                
                    elif self.exercise_type == 'Leg Curl':
                        # Calculate angle for the leg
                        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                   landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                        left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                        
                        angles['left'] = self.calculate_angle(left_hip, left_knee, left_ankle)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_hip[0] * w), int(left_hip[1] * h)),
                               (int(left_knee[0] * w), int(left_knee[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(left_knee[0] * w), int(left_knee[1] * h)),
                               (int(left_ankle[0] * w), int(left_ankle[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if angles['left'] > 160:
                                self.stage = "extended"
                            if angles['left'] < 80 and self.stage == "extended":
                                self.stage = "curled"
                                self.counter += 1
                                
                    elif self.exercise_type == 'Leg Extension':
                        # Calculate angle for the leg
                        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                   landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                        left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                        
                        angles['left'] = self.calculate_angle(left_hip, left_knee, left_ankle)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_hip[0] * w), int(left_hip[1] * h)),
                               (int(left_knee[0] * w), int(left_knee[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(left_knee[0] * w), int(left_knee[1] * h)),
                               (int(left_ankle[0] * w), int(left_ankle[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if angles['left'] < 90:
                                self.stage = "bent"
                            if angles['left'] > 160 and self.stage == "bent":
                                self.stage = "extended"
                                self.counter += 1
                                
                    elif self.exercise_type == 'Abdominal Crunch':
                        # Calculate angle for the torso
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                        left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                                   landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                        
                        angles['left'] = self.calculate_angle(left_shoulder, left_hip, left_knee)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_shoulder[0] * w), int(left_shoulder[1] * h)),
                               (int(left_hip[0] * w), int(left_hip[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(left_hip[0] * w), int(left_hip[1] * h)),
                               (int(left_knee[0] * w), int(left_knee[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if angles['left'] > 120:
                                self.stage = "down"
                            if angles['left'] < 80 and self.stage == "down":
                                self.stage = "up"
                                self.counter += 1
                                
                    elif self.exercise_type == 'Chest Fly':
                        # Track wrist positions relative to shoulders
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                    landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
                        
                        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                        right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                                     landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
                        
                        # Calculate distance between wrists for tracking the fly movement
                        wrist_distance = np.sqrt((left_wrist[0] - right_wrist[0])**2 + 
                                               (left_wrist[1] - right_wrist[1])**2)
                        
                        # Draw movement lines
                        cv2.line(image, (int(left_shoulder[0] * w), int(left_shoulder[1] * h)),
                               (int(left_wrist[0] * w), int(left_wrist[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(right_shoulder[0] * w), int(right_shoulder[1] * h)),
                               (int(right_wrist[0] * w), int(right_wrist[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if wrist_distance > 0.6:  # Arms wide apart
                                self.stage = "open"
                            if wrist_distance < 0.2 and self.stage == "open":  # Arms close together
                                self.stage = "closed"
                                self.counter += 1
                                
                    elif self.exercise_type == 'Shrugs':
                        # Track shoulder positions relative to ears
                        left_ear = [landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].x,
                                  landmarks[mp_pose.PoseLandmark.LEFT_EAR.value].y]
                        left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                                       landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
                        
                        right_ear = [landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].x,
                                   landmarks[mp_pose.PoseLandmark.RIGHT_EAR.value].y]
                        right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                                        landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
                        
                        # Calculate distance between ears and shoulders
                        left_distance = np.sqrt((left_ear[0] - left_shoulder[0])**2 + 
                                              (left_ear[1] - left_shoulder[1])**2)
                        right_distance = np.sqrt((right_ear[0] - right_shoulder[0])**2 + 
                                               (right_ear[1] - right_shoulder[1])**2)
                        
                        avg_distance = (left_distance + right_distance) / 2
                        
                        # Draw tracking lines
                        cv2.line(image, (int(left_ear[0] * w), int(left_ear[1] * h)),
                               (int(left_shoulder[0] * w), int(left_shoulder[1] * h)), (0, 255, 0), 2)
                        cv2.line(image, (int(right_ear[0] * w), int(right_ear[1] * h)),
                               (int(right_shoulder[0] * w), int(right_shoulder[1] * h)), (0, 255, 0), 2)
                        
                        self.form_feedback = self.validate_form(self.exercise_type, landmarks, angles)
                        
                        if not self.form_feedback:
                            if avg_distance > 0.15:  # Shoulders low
                                self.stage = "down"
                            if avg_distance < 0.11 and self.stage == "down":  # Shoulders raised
                                self.stage = "up"
                                self.counter += 1
                    
                    # Record positions for trajectory analysis if needed
                    if self.exercise_type in ['Bicep Curl', 'Lateral Raise', 'Front Raise', 'Shoulder Press']:
                        movement_buffer.append([landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                                             landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y])
                        if len(movement_buffer) > buffer_size:
                            movement_buffer.pop(0)
                            
                        # Draw movement path for better visual feedback
                        if len(movement_buffer) > 1:
                            for i in range(1, len(movement_buffer)):
                                start_point = (int(movement_buffer[i-1][0] * w), int(movement_buffer[i-1][1] * h))
                                end_point = (int(movement_buffer[i][0] * w), int(movement_buffer[i][1] * h))
                                cv2.line(overlay, start_point, end_point, (255, 0, 0), 2)
                                
                except Exception as e:
                    print(f"Error processing landmarks: {e}")
                    pass
                
                # Add partially transparent overlay for movement trajectory
                alpha = 0.4
                cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0, image)
                
                # Draw pose landmarks
                mp_draw.draw_landmarks(
                    image, 
                    results.pose_landmarks, 
                    mp_pose.POSE_CONNECTIONS,
                    mp_draw.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=2),
                    mp_draw.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                )
                
                # Display exercise info
                cv2.rectangle(image, (0, 0), (400, 120), (245, 117, 16), -1)
                
                # Display exercise type
                cv2.putText(image, f'Exercise: {self.exercise_type}', 
                          (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
                
                # Display counter
                cv2.putText(image, f'Count: {self.counter}', 
                          (15, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
                
                # Display stage
                cv2.putText(image, f'Stage: {self.stage}', 
                          (15, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
                
                # Display form feedback if any
                if self.form_feedback:
                    cv2.rectangle(image, (0, 450), (640, 480), (0, 0, 255), -1)
                    cv2.putText(image, f'Form: {self.form_feedback}', 
                              (15, 470), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Show the frame
            cv2.imshow(window_name, image)
            
            # Break the loop on 'q' press
            if cv2.waitKey(10) & 0xFF == ord('q'):
                self.running = False
                break
                
        cap.release()
        cv2.destroyAllWindows()
        # Update the UI counter after closing the camera
        self.update_counter_label()

    def create_main_window(self):
        """Create the main tkinter window for exercise selection"""
        self.root = Tk()
        self.root.title("Exercise Tracker")
        self.root.geometry("800x600")
        
        # Apply a modern style
        style = Style()
        style.configure("TFrame", background="#f0f0f0")
        style.configure("TButton", font=('Arial', 12), background="#4CAF50", foreground="white")
        style.configure("TLabel", font=('Arial', 12), background="#f0f0f0")
        style.configure("Header.TLabel", font=('Arial', 16, 'bold'), background="#f0f0f0")
        style.configure("Counter.TLabel", font=('Arial', 40, 'bold'), background="#f0f0f0")
        
        # Create main frame
        main_frame = ttk.Frame(self.root, padding="20")
        main_frame.pack(fill=BOTH, expand=True)
        
        # Create header
        header_label = ttk.Label(main_frame, text="Exercise Form Tracker", style="Header.TLabel")
        header_label.pack(pady=(0, 20))
        
        # Create exercise selection frame
        selection_frame = ttk.Frame(main_frame)
        selection_frame.pack(fill=X, pady=10)
        
        exercise_label = ttk.Label(selection_frame, text="Select Exercise:")
        exercise_label.pack(side=LEFT, padx=(0, 10))
        
        # Create the dropdown for exercise selection
        self.exercise_var = StringVar()
        exercise_options = list(self.exercise_info.keys())
        exercise_dropdown = ttk.Combobox(selection_frame, textvariable=self.exercise_var, 
                                       values=exercise_options, width=20)
        exercise_dropdown.current(0)  # Default to first exercise
        exercise_dropdown.pack(side=LEFT)
        exercise_dropdown.bind("<<ComboboxSelected>>", self.on_exercise_select)
        
        # Create counter display
        counter_frame = ttk.Frame(main_frame)
        counter_frame.pack(fill=X, pady=20)
        
        counter_label = ttk.Label(counter_frame, text="Reps:", style="Header.TLabel")
        counter_label.pack(side=LEFT, padx=(0, 10))
        
        self.counter_value = ttk.Label(counter_frame, text="0", style="Counter.TLabel")
        self.counter_value.pack(side=LEFT)
        
        # Create description frame
        desc_frame = ttk.Frame(main_frame)
        desc_frame.pack(fill=X, pady=10)
        
        desc_label = ttk.Label(desc_frame, text="Description:", style="Header.TLabel")
        desc_label.pack(anchor=W)
        
        self.desc_text = Text(desc_frame, height=3, width=60, wrap=WORD, font=('Arial', 11))
        self.desc_text.pack(fill=X, pady=5)
        self.desc_text.config(state=DISABLED)
        
        # Create position guidance frame
        position_frame = ttk.Frame(main_frame)
        position_frame.pack(fill=X, pady=10)
        
        position_label = ttk.Label(position_frame, text="Position:", style="Header.TLabel")
        position_label.pack(anchor=W)
        
        self.position_text = Text(position_frame, height=1, width=60, wrap=WORD, font=('Arial', 11))
        self.position_text.pack(fill=X, pady=5)
        self.position_text.config(state=DISABLED)
        
        # Create steps frame
        steps_frame = ttk.Frame(main_frame)
        steps_frame.pack(fill=X, pady=10)
        
        steps_label = ttk.Label(steps_frame, text="Steps:", style="Header.TLabel")
        steps_label.pack(anchor=W)
        
        self.steps_text = Text(steps_frame, height=5, width=60, wrap=WORD, font=('Arial', 11))
        self.steps_text.pack(fill=X, pady=5)
        self.steps_text.config(state=DISABLED)
        
        # Create buttons frame
        buttons_frame = ttk.Frame(main_frame)
        buttons_frame.pack(fill=X, pady=20)
        
        # Start button
        start_button = ttk.Button(buttons_frame, text="Start Tracking", command=self.start_tracking)
        start_button.pack(side=LEFT, padx=10)
        
        # Reset button
        reset_button = ttk.Button(buttons_frame, text="Reset Counter", command=self.reset_counter)
        reset_button.pack(side=LEFT, padx=10)
        
        # Exit button
        exit_button = ttk.Button(buttons_frame, text="Exit", command=self.exit_app)
        exit_button.pack(side=LEFT, padx=10)
        
        # Set initial exercise
        self.on_exercise_select(None)
        
        # Start the main loop
        self.root.protocol("WM_DELETE_WINDOW", self.exit_app)
        self.root.mainloop()
    
    def on_exercise_select(self, event):
        """Handle exercise selection"""
        exercise = self.exercise_var.get()
        self.exercise_type = exercise
        
        # Update description text
        self.desc_text.config(state=NORMAL)
        self.desc_text.delete(1.0, END)
        self.desc_text.insert(END, self.exercise_info[exercise]['description'])
        self.desc_text.config(state=DISABLED)
        
        # Update position text
        self.position_text.config(state=NORMAL)
        self.position_text.delete(1.0, END)
        self.position_text.insert(END, self.exercise_info[exercise]['position'])
        self.position_text.config(state=DISABLED)
        
        # Update steps text
        self.steps_text.config(state=NORMAL)
        self.steps_text.delete(1.0, END)
        for step in self.exercise_info[exercise]['steps']:
            self.steps_text.insert(END, step + "\n")
        self.steps_text.config(state=DISABLED)
    
    def start_tracking(self):
        """Start the exercise tracking with camera"""
        if self.exercise_type:
            # Reset counter when starting new tracking session
            self.counter = 0
            self.stage = None
            self.form_feedback = ""
            self.running = True
            
            # Start camera in a separate thread
            import threading
            camera_thread = threading.Thread(target=self.start_camera)
            camera_thread.daemon = True
            camera_thread.start()
    
    def reset_counter(self):
        """Reset the exercise counter"""
        self.counter = 0
        self.update_counter_label()
    
    def update_counter_label(self):
        """Update the counter label in the UI"""
        self.counter_value.config(text=str(self.counter))
    
    def exit_app(self):
        """Exit the application"""
        self.running = False
        self.root.destroy()

if __name__ == "__main__":
    app = ExerciseCounter()